In [1]:
from pathlib import Path
import json
import utilities.utilities as util
import pandas as pd

import os
from dotenv import load_dotenv, find_dotenv

/home/jovyan/MasterThesis/mahjong_project/utilities/utilities.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange, tqdm


In [2]:
load_dotenv(find_dotenv('config.env'))
JSON_LOGS_PATH = Path(os.environ.get('JSON_DATASET'))
JSON_LOGS_PATH

PosixPath('/home/jovyan/MasterThesis/data/json_logs')

In [3]:
OUTPUT_PATH = Path.home() / 'MasterThesis' / 'data' / 'state_data_new'
OUTPUT_PATH.mkdir(exist_ok=True)
OUTPUT_PATH

PosixPath('/home/jovyan/MasterThesis/data/state_data_new')

In [4]:
INIT = 'Init'
DRAW = 'Draw'
DISCARD = 'Discard'
CHI = 'Chi'
PON = 'Pon'
MINKAN = 'MinKan'  # Open Kan
ANKAN = 'AnKan'  # Closed Kan
KAKAN = 'KaKan'  # Added Kan
NUKI = 'Nuki'  # Declare North dora
REMOVE = 'Remove'
REACH = 'Riichi'

In [5]:
years = util.get_all_logs_annually(JSON_LOGS_PATH,
                                   years=[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019],
                                   progress_bar=True,
                                   filterblade_path=Path('log_game_data.parquet'))

for year, logs in years:

    output_path = OUTPUT_PATH / str(year)
    output_path.mkdir(parents=True, exist_ok=True)

    for log_json in logs:

        rows = []  # List of Dictionaries

        log = json.load(log_json.open())

        for round_number, actions in enumerate(log['rounds']):

            # STEP 1: Handle INIT case in isolation from the rest
            # Rationale:
            #   A log always begin with INIT, isolate it
            #   such that next step can forgo an single-time-use if-statement

            init = actions.pop(0)

            init_data = init['data']

            rows.append({
                'log_id': log_json.stem,
                'round': round_number,
                'step': 0,
                'player': -1,
                'action': INIT,
                'tile': 255
            })

            for player, starting_hand in enumerate(init_data['hands']):
                for tile in starting_hand:
                    rows.append({
                        'log_id': log_json.stem,
                        'round': round_number,
                        'step': 0,
                        'player': player,
                        'action': DRAW,
                        'tile': tile
                    })

            rows.append({
                'log_id': log_json.stem,
                'round': round_number,
                'step': 0,
                'player': -1,
                'action': 'Dora',
                'tile': init_data['dora']  # Tile revealed
            })

            # STEP 2: Handle actions after INIT
            # Loop Params
            pons = [set(), set(), set(), set()]  # Needed for Chankan = Robbing a Kan
            previous_kakan_tile = -1  # Needed for Chankan = Robbing a Kan
            discard_tile = -1  # Needed when removing from pool
            step_offset = 1
            next_discard_is_reach = False

            for step, action in enumerate(actions):

                tag = action['tag']

                if tag == 'DRAW':

                    previous_drawn_tile = action['data']['tile']

                    rows.append({
                        'log_id': log_json.stem,
                        'round': round_number,
                        'step': step + step_offset,
                        'player': action['data']['player'],
                        'action': DRAW,
                        'tile': previous_drawn_tile
                    })

                elif tag == 'DISCARD':

                    discard_tile = action['data']['tile']  # Store discarded tile in history

                    rows.append({
                        'log_id': log_json.stem,
                        'round': round_number,
                        'step': step + step_offset,
                        'player': action['data']['player'],
                        'action': DISCARD if not next_discard_is_reach else REACH,  # Merged Discard with Reach
                        'tile': discard_tile  # Previous discard pile
                    })

                    next_discard_is_reach = False

                elif tag == 'CALL':

                    meld = action['data']['mentsu']  # List of classes, not sorted
                    call_type = action['data']['call_type']

                    if call_type in [CHI, PON, MINKAN]:  # The 3 calls that removes from callees pool
                        rows.append({
                            'log_id': log_json.stem,
                            'round': round_number,
                            'step': step + step_offset,
                            'player': action['data']['callee'],
                            'action': REMOVE,
                            'tile': discard_tile  # Previous discard pile
                        })

                    # Handles Chankan - Robbing a Kan
                    if call_type == PON:
                        pons[action['data']['caller']].update(meld)
                    elif call_type == KAKAN:
                        added_tile = list(set(meld) - pons[action['data']['caller']])
                        assert len(added_tile) == 1, 'MORE THAN ONE ADDED TILE!'
                        previous_kakan_tile = added_tile[0]
                    
                    # Handles Ankan - NOTE: Ankans have only 2 tiles in its meld in the original data. We re-populate it to 4.
                    if call_type == ANKAN:
                        base_tile_class = (meld[0] // 4) * 4  # Not important which tile of the meld we pick here => therefore retrieving the first element
                        meld = [base_tile_class + i for i in range(4)]

                    for tile in sorted(meld):  # Added sorting here to include more predictability
                        rows.append({
                            'log_id': log_json.stem,
                            'round': round_number,
                            'step': step + step_offset,
                            'player': action['data']['caller'],
                            'action': action['data']['call_type'],
                            'tile': tile
                        })

                elif tag == 'REACH' and action['data']['step'] == 1:  # 'REACH' is two-step, we check only for 1st part
                    next_discard_is_reach = True
                    step_offset -= 1

                elif tag == 'DORA':

                    step_offset -= 1  # Sync with prior Kan

                    rows.append({
                        'log_id': log_json.stem,
                        'round': round_number,
                        'step': step + step_offset,
                        'player': -1,
                        'action': 'Dora',
                        'tile': action['data']['hai']  # Tile revealed
                    })

                elif tag == 'AGARI':

                    is_ron = 'loser' in action['data']  # Check if RON or TSUMO
                    previous_action = rows[-1]['action']

                    if is_ron:  # if RON -> prepend REMOVE action
                        rows.append({
                            'log_id': log_json.stem,
                            'round': round_number,
                            'step': step + step_offset,
                            'player': action['data']['loser'],
                            'action': REMOVE,
                            'tile': discard_tile if previous_action != KAKAN else previous_kakan_tile
                            # Previous discard pile
                        })

                    rows.append({
                        'log_id': log_json.stem,
                        'round': round_number,
                        'step': step + step_offset,
                        'player': action['data']['winner'],
                        'action': 'Ron' if is_ron else 'Tsumo',
                        'tile': rows[-1]['tile'] if previous_action != KAKAN else previous_kakan_tile
                        # Previous tile (either from DISCARD or DRAW or KAKAN)
                    })

                elif tag == 'RYUUKYOKU':
                    rows.append({
                        'log_id': log_json.stem,
                        'round': round_number,
                        'step': step + step_offset,
                        'player': -1,
                        'action': 'Ryuukyoku',
                        'tile': 255  # Tile revealed
                    })

        df = pd.DataFrame(rows)

        # Managing dtypes (This includes a noticeable overhead about 10-15% in time)
        df = df.astype({
            'log_id': pd.StringDtype(),
            'round': 'uint8',
            'step': 'uint8',
            'player': 'int8',
            'action': 'category',  # dtype `category` will not be saved, and will later be read as `object`
            'tile': 'uint8'
        })

        df.to_parquet(output_path / f"{log_json.stem}.parquet", engine='fastparquet')

2014:   0%|          | 0/141132 [00:00<?, ?it/s]

2018:   0%|          | 0/173548 [00:00<?, ?it/s]

2017:   0%|          | 0/169384 [00:00<?, ?it/s]

2019:   0%|          | 0/171629 [00:00<?, ?it/s]

2009:   0%|          | 0/6895 [00:00<?, ?it/s]

...

2016:   0%|          | 0/159665 [00:00<?, ?it/s]

2012:   0%|          | 0/116985 [00:00<?, ?it/s]

2010:   0%|          | 0/70228 [00:00<?, ?it/s]

2015:   0%|          | 0/152280 [00:00<?, ?it/s]

2011:   0%|          | 0/100814 [00:00<?, ?it/s]